In [75]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import random
import json
import io
import os
from tqdm.notebook import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [76]:
with open("rotater_config.json", "r") as f:
    config = json.load(f)

def rotater():
    # rotate configuration
    user_agent = random.choice(config["user_agents"])
    proxy = config["proxy"]
    delay = random.uniform(0, 1)
    
    headers = {'User-Agent': user_agent}
    proxies={"http":proxy, "https":proxy}
    
    return headers, proxies, delay

def get_bs_object(url, use_proxy=True, parser='html.parser'):
    if use_proxy:
        headers, proxies, delay = rotater()

        page = requests.get(url, headers=headers, proxies=proxies, verify=False)
    else:
        page = requests.get(url, verify=False)
    soup = bs(page.text, parser)
    return soup

## get industries

In [77]:
industries = "Basic Materials, Communication Services, Consumer Cyclical, Consumer Defensive, Energy, Financial Services, Healthcare, Industrials, Real Estate, Technology, Utilities".split(", ")
href_ind = "https://finance.yahoo.com/sector/ms_basic_materials, https://finance.yahoo.com/sector/ms_communication_services, https://finance.yahoo.com/sector/ms_consumer_cyclical, https://finance.yahoo.com/sector/ms_consumer_defensive, https://finance.yahoo.com/sector/ms_energy, https://finance.yahoo.com/sector/ms_financial_services, https://finance.yahoo.com/sector/ms_healthcare, https://finance.yahoo.com/sector/ms_industrials, https://finance.yahoo.com/sector/ms_real_estate, https://finance.yahoo.com/sector/ms_technology, https://finance.yahoo.com/sector/ms_utilities".split(", ")
industries_dict = dict(zip(industries, href_ind))

In [78]:
industries_dict

{'Basic Materials': 'https://finance.yahoo.com/sector/ms_basic_materials',
 'Communication Services': 'https://finance.yahoo.com/sector/ms_communication_services',
 'Consumer Cyclical': 'https://finance.yahoo.com/sector/ms_consumer_cyclical',
 'Consumer Defensive': 'https://finance.yahoo.com/sector/ms_consumer_defensive',
 'Energy': 'https://finance.yahoo.com/sector/ms_energy',
 'Financial Services': 'https://finance.yahoo.com/sector/ms_financial_services',
 'Healthcare': 'https://finance.yahoo.com/sector/ms_healthcare',
 'Industrials': 'https://finance.yahoo.com/sector/ms_industrials',
 'Real Estate': 'https://finance.yahoo.com/sector/ms_real_estate',
 'Technology': 'https://finance.yahoo.com/sector/ms_technology',
 'Utilities': 'https://finance.yahoo.com/sector/ms_utilities'}

In [82]:
def get_symbol(industry_url, length=25):
    headers, proxies, delay = rotater()
    s_ind = requests.get(industry_url, proxies=proxies, verify=False).text
    df_ind = pd.read_html(s_ind)[0]
    return list(df_ind["Symbol"].values[:length])

def get_csv_file(industry, symbol):
    headers, proxies, delay = rotater()
    url = f"https://finance.yahoo.com/quote/{symbol}/history?p={symbol}"
    s_sym = requests.get(url, proxies=proxies, verify=False).text
    df_sym = pd.read_html(s_sym)[0]
    
    df_sym.to_csv(os.path.join(f"results/{industry}_{symbol}.csv".replace(" ","_").lower()), index=False)

In [83]:
for ind in industries_dict.keys():
    symbols = get_symbol(industries_dict[ind])
    print(f"process industry {ind}")
    for sym in tqdm(symbols):
        get_csv_file(ind, sym)

process industry Basic Materials



process industry Communication Services



process industry Consumer Cyclical



process industry Consumer Defensive



process industry Energy


...


process industry Healthcare



process industry Industrials



process industry Real Estate



process industry Technology



process industry Utilities
